In [1]:
from langgraph.graph import StateGraph
from typing import TypedDict

In [2]:
class BatsmanState(TypedDict):
    runs: int
    balls: int
    fours: int
    sixes: int

    sr: float
    bpb: float
    boundary_percent: float

In [ ]:
def calculate_sr(state: BatsmanState) -> BatsmanState:
    runs = state['runs']
    balls = state['balls']

    state['sr'] = runs/balls * 100

    return state

def calculate_bpb(state: BatsmanState) -> BatsmanState:
    fours = state['fours']
    sixes = state['sixes']
    balls = state['balls']

    state['bpb'] = balls/(fours+sixes)

    return state

def calculate_bp(state: BatsmanState) -> BatsmanState:      
    fours = state['fours']
    sixes = state['sixes']
    runs = state['runs']

    state['boundary_percent'] = ((fours*4)+(sixes*6))/runs

    return state


In [ ]:
graph = StateGraph(BatsmanState)

graph.add_node("calculate_sr",calculate_sr)
graph.add_node("calculate_bpb",calculate_bpb)
graph.add_node("calculate_bp",calculate_bp)
graph.add_node("summary",summary)
